<a href="https://colab.research.google.com/github/neermesh/dataPreprocessing/blob/master/data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pandas_datareader import data as pdr
data = pdr.get_data_yahoo('ASIANPAINT.NS', start='2020-01-01', end='2020-01-24')  

In [0]:
data

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-01-01,1802.550049,1783.199951,1790.199951,1793.199951,397285,1793.199951
2020-01-02,1799.900024,1783.400024,1791.000000,1790.650024,411266,1790.650024
2020-01-03,1779.800049,1747.050049,1779.800049,1751.400024,1318628,1751.400024
2020-01-06,1738.000000,1694.000000,1737.949951,1707.150024,1871804,1707.150024
2020-01-07,1740.500000,1711.000000,1711.000000,1724.400024,1033650,1724.400024
2020-01-08,1736.000000,1695.000000,1696.000000,1728.800049,1006057,1728.800049
2020-01-09,1775.000000,1741.000000,1763.000000,1772.550049,1781733,1772.550049
2020-01-10,1797.300049,1760.000000,1772.000000,1792.550049,1256612,1792.550049
2020-01-13,1807.949951,1794.349976,1799.000000,1804.900024,823821,1804.900024
